In [44]:
import os
import argparse
import numpy as np
from time import time
from collections import defaultdict

BASE_LOC = r'/home/or/dev/Intro_to_NLP/task2'
POS_DATA_LOC = os.path.join(BASE_LOC, r'data/pos')
POS_TRAIN_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-train')
POS_DEV_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-dev')
#POS_TEST_FILE = os.path.join(POS_DATA_LOC, 'ass1-tagger-test-input')

In [47]:
def add_wp_to_dict(the_dict, word, pos):
    try:
        if word in the_dict:
            if pos in the_dict[word]:
                the_dict[word][pos]['wp_count'] += 1
            else:
                the_dict[word][pos] = {'wp_count':1, 'right_pos':{}, 'left_pos':{}}

        else:
            the_dict[word] = {pos: {'wp_count':1, 'right_pos':{}, 'left_pos':{}}}
    except:
        import pdb; pdb.set_trace();

def add_right_pos_to_dict(the_dict, word, pos, right_pos):
    try:
        #import pdb; pdb.set_trace();
        if right_pos in the_dict[word][pos]['right_pos']:
            the_dict[word][pos]['right_pos'][right_pos]['right_count'] += 1
        else:
            the_dict[word][pos]['right_pos'][right_pos] = {'right_count':1, 'left_pos':{}}
    except:
        import pdb; pdb.set_trace();

def add_left_pos_to_dict(the_dict, word, pos, left_pos):
    if left_pos in the_dict[word][pos]['left_pos']:
        the_dict[word][pos]['left_pos'][left_pos]['left_count'] += 1
    else:
        the_dict[word][pos]['left_pos'][left_pos] = {'left_count':1}
        
def add_wp_right_pos_left_pos_to_dict(the_dict, word, pos, right_pos, left_pos):
    try:    
        if left_pos in the_dict[word][pos]['right_pos'][right_pos]['left_pos']:
            the_dict[word][pos]['right_pos'][right_pos]['left_pos'][left_pos]['left_count'] += 1
        else:
            the_dict[word][pos]['right_pos'][right_pos]['left_pos'][left_pos] = {'left_count':1} 
    except:
        import pdb; pdb.set_trace();        
def split_token_pos(token_and_pos, loc):
    try:    
        token, pos = token_and_pos[loc].rsplit('/',1)
    except:
        import pdb; pdb.set_trace()
    return token, pos



def read_file(loc):
    file = open(POS_TRAIN_FILE, "r")
    file_lines = file.readlines()
    file.close()
    return file_lines

def create_dict():
    file_lines = read_file(POS_TRAIN_FILE)
    words_pos_dict = defaultdict()
    for file_line in file_lines:
        splitted_line = file_line.split()
        len_line = len(splitted_line)
        for ii, _ in enumerate(splitted_line):
            token, pos = split_token_pos(splitted_line, ii)
            add_wp_to_dict(words_pos_dict, token, pos)
            if ii  + 1 < len_line:
                right_token, right_pos = split_token_pos(splitted_line, ii + 1)
                add_right_pos_to_dict(words_pos_dict, token, pos, right_pos)
            if ii - 1 <= 0:
                left_token, left_pos = split_token_pos(splitted_line, ii - 1)
                add_left_pos_to_dict(words_pos_dict, token, pos, left_pos)
            if (ii - 1 <= 0) and (ii  + 1 < len_line):
                add_wp_right_pos_left_pos_to_dict(words_pos_dict, token, pos, right_pos, left_pos)
    return words_pos_dict

    file_lines = read_file(POS_TRAIN_FILE)

words_pos_dict = create_dict()
read_file(loc)
#POS_DEV_FILE

In [48]:
file_lines = read_file(POS_DEV_FILE)

In [142]:
def possible_location(annotated_location):
    len_line = len(annotated_location)
    dist_tokens = np.ones(len_line) * -1

    for loc in range(len_line):
        if annotated_location[loc] == 0:
            loc_value = 0
            if loc + 1 < len_line:
                if annotated_location[loc + 1] == 1:
                    loc_value += 2

            if loc - 1 >= 0:
                if annotated_location[loc -1] == 1:
                    loc_value += 1

            dist_tokens[loc] = loc_value
    print(dist_tokens)
    return dist_tokens

def split_to_lists(line):
    words_list =[]
    pos_list = []
    
    for token_and_pos in line.split():
        token, pos = token_and_pos.rsplit('/',1)
        words_list.append(token)
        pos_list.append(pos)
        
    return words_list, pos_list

In [ ]:
def match_value(the_dict, indication, right_pos=None,left_pos=None):
    gotten_pos_val = NONE_EXIST
    if indication == BOTH_POS_IND:
        if right_pos in the_dict['right_pos'] and \
        left_pos in the_dict['right_pos'][right_pos]['left_pos']:
            gotten_pos_val = the_dict['right_pos'][right_pos]['left_pos'][left_pos]['left_count']
        
    if indication == RIGHT_POS_IND:
        if right_pos in the_dict['right_pos']:
            gotten_pos_val = the_dict['right_pos'][right_pos]['right_count']
        
    if indication == LEFT_POS_IND:
        if left_pos in the_dict['left_pos']:
            gotten_pos_val = the_dict['left_pos'][left_pos]

    if indication == NONE_POS_IND:
        gotten_pos_val = the_dict['wp_count']
        
    return gotten_pos_val
    
NONE_EXIST = -1
NONE_POS_IND = 0
LEFT_POS_IND = 1
RIGHT_POS_IND = 2
BOTH_POS_IND = 3


    
def test_both(loc, the_dict, words_list, pos_list, indicaton):
    best_pos = ''
    best_count = NONE_EXIST
    
    if words_list[loc] in the_dict.keys():
        right_pos = pos_list[loc + 1]
        left_pos = pos_list[loc - 1]
        
        for pos, pos_values in the_dict[words_list[loc]].items():
            val = match_value(the_dict, indication, right_pos=None,left_pos=None)
            if best_count < val:
                best_count = val]
                best_pos = pos

    return best_count, best_pos

def test_right(loc, the_dict, words_list, pos_list):
    if words_list[loc] in the_dict.keys():
        right_pos = pos_list[loc + 1]
        if right_pos in words_list[loc]['right_pos']:
            gotten_right_pos = words_list[loc]['right_pos'][right_pos]
            return gotten_right_pos.keys()[0], gotten_right_pos['right_count']
        
    return NONE_EXIST, _

# def test_both(loc, the_dict, words_list, pos_list):
#     if words_list[loc] in the_dict.keys():
#         left_pos = pos_list[loc - 1]
#         if left_pos in words_list[loc]['left_pos']:
#             gotten_left_pos = words_list[loc]['left_pos'][left_pos]
#             return gotten_left_pos.keys()[0], gotten_left_pos['left_count']
        
#     return NONE_EXIST, _

# def test_both(loc, the_dict, words_list, pos_list):
#     if words_list[loc] in the_dict.keys():
#         right_pos = pos_list[loc + 1]
#         left_pos = pos_list[loc - 1]
#         if right_pos in words_list[loc]['right_pos'] and \
#         left_pos in words_list[loc]['right_pos'][right_pos]['left_pos']:
#             left_pos =words_list[loc]['right_pos'][right_pos]['left_pos']['left_pos']
#             return left_pos.keys()[0], left_pos['left_count']
        
#     return NONE_EXIST, _

In [144]:
words_list, pos_list = split_to_lists(file_lines[0])
a =[1,1,0, 0]
all_possible_locs = possible_location(a)

possible_both_locs = np.where(possible_locs == BOTH_POS_IND)
possible_right_locs = np.where(possible_locs == RIGHT_POS_IND)
possible_left_locs = np.where(possible_locs == LEFT_POS_IND)
possible_none_locs = np.where(possible_locs == NONE_POS_IND)

best_both_pos = ''
best_both_count = NONE_EXIST
best_right_pos = ''
best_right_count = NONE_EXIST
best_left_pos = ''
best_left_count = NONE_EXIST
best_none_pos = ''
best_none_count = NONE_EXIST

# for index in possible_both_locs:
#     if possoble_locs[index] == BOTH_POS_IND:
#         tested_pos, tested_val = test_both()
#         if tested_pos != NONE_EXIST:
#             if  best_both_count < tested_val:
#                 best_both_count = tested_val
#                 best_both_pos = tested_pos
                
# if best_both_count != NONE_EXIST:
#     return best_both_pos

best_count, best_pos = get_pos(possible_locs, test_both, BOTH_POS_IND, the_dict, words_list, pos_list)
if best_count != NONE_EXIST
    return best_pos
best_count, best_pos = get_pos(possible_locs, test_func, RIGHT_POS_IND, the_dict, words_list, pos_list)
if best_count != NONE_EXIST:
    return best_pos
best_count, best_pos = get_pos(possible_locs, test_func, LEFT_POS_IND, the_dict, words_list, pos_list)
if best_count != NONE_EXIST:
    return best_pos
best_count, best_pos = get_pos(possible_locs, test_func, NONE_POS_IND, the_dict, words_list, pos_list)
return best_pos

def get_pos(all_possible_locs, test_func, indication, the_dict, words_list, pos_list):
    possible_both_locs = np.where(all_possible_locs == indication)

    best_pos = ''
    best_count = NONE_EXIST

    for loc in possible_locs:
        tested_pos, tested_val = test_func(loc, the_dict, words_list, pos_list)
        if tested_pos != NONE_EXIST:
            if  best_count < tested_val:
                best_count = tested_val
                best_pos = tested_pos

    return best_count, best_pos

[-1. -1.  1.  0.]


array([2, 3, 1, 0])

In [113]:
def location_to_anottate(annotated_location):
    len_line = len(annotated_location)
    dist_tokens = np.ones(len_line) * -1

    for loc in range(len_line):
        if annotated_location[loc] == 0:
            loc_value = 0
            if loc + 1 < len_line:
                if annotated_location[loc + 1] == 1:
                    loc_value += 2

            if loc - 1 >= 0:
                if annotated_location[loc -1] == 1:
                    loc_value += 1

            if loc_value == 3:
                return loc, loc_value

            dist_tokens[loc] = loc_value

    index = np.argmax(dist_tokens)
    val_max = dist_tokens[index]
    return index, val_max

In [111]:
annotated_location

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [145]:
NONE_EXIST = -1
NONE_POS_IND = 0
LEFT_POS_IND = 1
RIGHT_POS_IND = 2
BOTH_POS_IND = 3



In [ ]:
if kind == NONE_POS_IND:
    if token in words_pos_dict.keys():
        max_count = -np.inf
        selected_pos = ''
        for tested_pos, pos_values in words_pos_dict[token].items():
            wp_count = pos_values['wp_count']
            if wp_count > max_count:
                max_count = wp_count 
                selected_pos = tested_pos
    else:
        raise token
        
if kind == LEFT_POS_IND:
    if token in words_pos_dict.keys():
    max_count = -np.inf
    selected_pos = ''
    for tested_pos, pos_values in words_pos_dict[token].items():
        for
        wp_count = pos_values['wp_count']
        if wp_count > max_count:
            max_count = wp_count 
            selected_pos = tested_pos
    else:
        raise token

In [82]:
splitted_line = file_lines[0].split()
len_line = len(splitted_line)
annotated_location = np.zeros(len_line)
annotated_token = [''] * len_line
words_list =[]
pos_lost = []
while not all(annotated_location):
    loc_to_annotate, kind = location_to_anottate(annotated_location)
    token, pos = split_token_pos(splitted_line, ii)
    
for ii, _ in enumerate(splitted_line):
    token, pos = split_token_pos(splitted_line, ii)
    #print(words_pos_dict[token])
    if token in words_pos_dict.keys():
        max_count = -np.inf
        selected_pos = ''
        for tested_pos, pos_values in words_pos_dict[token].items():
            wp_count = pos_values['wp_count']
            if wp_count > max_count:
                max_count = wp_count 
                selected_pos = tested_pos
    else:
        raise token

In [73]:
splitted_line = file_lines[0].split()
len_line = len(splitted_line)
for ii, _ in enumerate(splitted_line):
    token, pos = split_token_pos(splitted_line, ii)
    #print(words_pos_dict[token])
    if token in words_pos_dict.keys():
        max_count = -np.inf
        selected_pos = ''
        for tested_pos, pos_values in words_pos_dict[token].items():
            wp_count = pos_values['wp_count']
            if wp_count > max_count:
                max_count = wp_count 
                selected_pos = tested_pos
    else:
        raise token

IN 1735
RB 1
NNP 3
RBR 1
DT 3142
, 1
NNP 317
NN 1
CD 100
NN 36
VB 21
VBP 1
IN 22925
RP 2
RB 2
`` 6967
DT 6795
NNP 37
VB 1
NN 3
'' 6787
IN 4361
RP 1
NNP 197
POS 8079
VBZ 1222
PRP 8
NNP 1
NNS 1
NNP 7
NNP 5
( 1153
`` 6967
VBN 1
NNS 1
VBP 1
VB 8
DT 41098
VBP 1
NNP 5
NN 1
JJ 2
NN 1
NNP 2
IN 14957
RP 173
RB 53
FW 2
RBR 1
NNP 1
NNP 132
NN 7
, 48723
'' 6787
NN 2
NNP 1
CC 1033
SYM 1
NNS 2
NNP 4
NNPS 2
) 1160
, 48723
DT 41098
VBP 1
NNP 5
NN 1
JJ 2
NN 125
IN 22925
RP 2
RB 2
NNP 4
, 48723
VBN 23
VBD 30
IN 4483
RP 10
RB 2
NNP 7
NNP 1
, 48723
VBD 3903
RB 6
VBN 17
VBD 52
IN 9282
TO 12916
NNP 2
NNP 3
. 39020


In [55]:
token

'.'

In [56]:
if token in words_pos_dict.key():
    max_count = 0 
    for pos_dict in words_pos_dict[word]:
        print(pos_dict,pos_dict.keys())
        wp_count = pos_dict['wp_count']
        if wp_count > max_count:
            max_count = wp_count 
else:
    raise token
    

dict_keys(['.'])

In [ ]:
for words_pos_dict

In [ ]:

def saving(file_loc, dest_loc):
    file = open(file_loc, "r")
    sentence = []
    sentences = []
    file_chunk = file.readlines(CHUNCK)
    
    tic = time()
    while file_chunk:
        for line in file_chunk:
            if line == '\n':
                sentences.append(sentence)
                sentence = []
            else:
                splitted_line = line.split()
                sentence.append((splitted_line[2], splitted_line[3]))
        file_chunk = file.readlines(CHUNCK)
        
    if len(sentence):
        sentences.append(sentence)
    file.close()
    toc = time()
    sentence_len = len(sentences)
    
    print('File reading time(minutes): ', round((toc- tic)/60, 2),'Sentences: ', sentence_len)
    
    tic = time()
    with open(dest_loc, "wb" ) as f:
        pickle.dump(sentences, f)
    toc = time()
    print('Saving time(minutes): ', round((toc- tic)/60, 2))